In [1]:
# Database connector 
import mysql.connector
from mysql.connector import Error

import pandas as pd
import re
import numpy as np

In [2]:
# Connect to the database
connection = mysql.connector.connect(
    host = 'elegbede.mysql.database.azure.com',
    port = 3306,
    user = '##',
    password = '##',
    database = 'cyclistic_db'
)

# Function to execute a SQL query 
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Successful")
    except Error as err:
        print(f"Error: '{err}'")

# Read and Run Query
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall() # Reads data from the database without making any changes to it.
        return result
    except Error as err:
        print(f'Error: "{err}"')

# Use DB
execute_query(connection, """USE cyclistic_db""")


Successful


In [3]:
# Read data from the database and store as a dataframe
combined_months = read_query(connection, "SELECT * FROM combined_months;")

# Convert to a dataframe
df_months = pd.DataFrame(combined_months)

# Set columns
df_months.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at', 
    'start_station_name', 'start_station_id', 'end_station_name', 
    'end_station_id', 'start_lat', 'start_lng', 
    'end_lat', 'end_lng', 'member_casual']

df_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5738812 entries, 0 to 5738811
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    float64       
 6   end_station_name    object        
 7   end_station_id      float64       
 8   start_lat           object        
 9   start_lng           object        
 10  end_lat             object        
 11  end_lng             object        
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(2), object(9)
memory usage: 569.2+ MB


### Data Preparation

Drop Duplicates

In [4]:
df = df_months

# Handle and drop duplicates
print('Number of duplicate ride_ids:',df['ride_id'].duplicated().sum())
df = df.drop_duplicates(subset=['ride_id'])
print('No. duplicates after dropping:', df['ride_id'].duplicated().sum())

Number of duplicate ride_ids: 209
No. duplicates after dropping: 0


Handle Null Rows

In [5]:
# Drop rows without start and end time information
df = df.dropna(subset= ['started_at','ended_at'])

df = df.sort_values(by=['start_lat','start_lng'], ascending=True)

# Forward fill the missing start_station_name and start_station_id
df[['start_station_name', 'start_station_id']] = df[['start_station_name', 'start_station_id']].ffill().bfill()

# Fill the end stations
df = df.sort_values(by=['end_lat','end_lng'], ascending=True)
# Forward fill the missing start_station_name and start_station_id
df[['end_station_name', 'end_station_id','end_lat','end_lng']] = df[['end_station_name', 'end_station_id','end_lat','end_lng']].ffill().bfill()

df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

Data Extraction

In [6]:
df_clean = df

# Convert to date time
df_clean['started_at'] = pd.to_datetime(df_clean['started_at'],format='%Y-%m-%d %H:%M:%S')
df_clean['ended_at'] = pd.to_datetime(df_clean['ended_at'],format='%Y-%m-%d %H:%M:%S')

# Find the ride duration in seconds
df_clean['ride_duration'] = (df_clean['ended_at']-df_clean['started_at']).dt.total_seconds()

Remove outliers

In [7]:
p25 = df_clean['ride_duration'].quantile(0.25) # 25th percentile
p50 = df_clean['ride_duration'].quantile(0.5) # 50th percentile
p75 = df_clean['ride_duration'].quantile(0.75) # 75th percentile

# Interquartile range
iqr = p75 - p25
print(f'Interquartile range: {iqr}')

# Lower Limit
lower_bound  = p25 - 1.5 * iqr
print(f'Lower Bound: {lower_bound}')

# Upper Limit
upper_bound = p75 + 1.5 * iqr
print(f'Upper Bound: {upper_bound}')

# Mean
print(f'Mean: {df_clean['ride_duration'].mean()}')

# Eliminate outliers above the upper bound and negative ride durations
df_clean = df_clean[(df_clean['ride_duration'] >= 0) & (df_clean['ride_duration'] <= upper_bound)]

print(f'No of data points after cleaning: {df_clean.shape[0]}')

Interquartile range: 915.0
Lower Bound: -962.5
Upper Bound: 2697.5
Mean: 1273.7119174442303
No of data points after cleaning: 5176939


Save the dataframe and export to MySQL Database

In [8]:
# Export
data = df_clean.sort_values(by=['started_at','ended_at']).reset_index(drop=True)

data['start_station_id'] = data['start_station_id'].astype(int)
data['end_station_id'] = data['end_station_id'].astype(int)

#### Calculate revenue generated from each ride
- Members:
Unlocking Fee = $0
Classic Bike  = $0.17/minute
Electric Bike = $0.17/minute
Docked Bike = $0.20/minute

- Casual:
Unlocking Fee = $1.35
Classic Bike  = $0.20/minute
Electric Bike = $0.35/minute
Docked Bike = $0.40/minute

In [10]:
def add_revenue(row):

    ride_duration_mins = row['ride_duration']/60

    if row['member_casual'] == 'casual':

        unlocking_fee = 1.35 # Casual unlocking fee
        if row['rideable_type'] == 'electric_bike':
            return unlocking_fee + 0.35 * ride_duration_mins
        elif row['rideable_type'] == 'classic_bike':
            return unlocking_fee + 0.20 * ride_duration_mins
        elif row['rideable_type'] == 'docked_bike':
            return unlocking_fee + 0.40 * ride_duration_mins
        else:
            return 0
        
    elif row['member_casual'] == 'member':
        if row['rideable_type'] == 'electric_bike':
            return 0.17 * ride_duration_mins
        elif row['rideable_type'] == 'classic_bike':
            return 0.17 * ride_duration_mins
        elif row['rideable_type'] == 'docked_bike':
            return 0.20 * ride_duration_mins
        else:
            return 0
    else:
        return None

data['revenue'] = data.apply(add_revenue, axis = 1)

data[['rideable_type','member_casual','ride_duration','revenue']].sample(10)
    

,rideable_type,member_casual,ride_duration,revenue
5138136,electric_bike,casual,139.0,2.160833
4881348,classic_bike,member,278.0,0.787667
3116841,classic_bike,casual,1607.0,6.706667
1961936,classic_bike,member,547.0,1.549833
2693966,electric_bike,member,210.0,0.595000
5008624,electric_bike,casual,385.0,3.595833
3430285,classic_bike,member,494.0,1.399667
1032124,electric_bike,member,347.0,0.983167
41925,docked_bike,member,623.0,2.076667
2023634,electric_bike,member,1630.0,4.618333


In [11]:
# Close connection
connection.close()